# BreezeAPI Options Strategy

## Install BreezeAPI package

In [ ]:
pip install --upgrade breeze-connect

In [ ]:
pip install --upgrade mplfinance

In [ ]:
pip install --upgrade matplotlib

In [ ]:
pip install --upgrade pandas_ta

## Import required packages

In [ ]:
from breeze_connect import BreezeConnect
from datetime import datetime
import urllib
import numpy as np
import pandas as pd

import mplfinance as mpf
import matplotlib.pyplot as plt
import pandas_ta as ta


## Variables for initial setup

In [ ]:
Api_key="^42y6975650632#94258t282@r718`77"
Api_Secret="i0td81HE339202156w96102D98497633"

## Get Link to generate session key

In [ ]:
print("https://api.icicidirect.com/apiuser/login?api_key="+urllib.parse.quote_plus(Api_key))

## Declare Session key variable

In [ ]:
Api_Session="39395901"

## Initialize SDK

In [ ]:
breeze = BreezeConnect(api_key=Api_key)

## Generate Session

In [ ]:
breeze.generate_session(api_secret=Api_Secret,
                        session_token=Api_Session)


## Indicator process

In [ ]:
currentDate=datetime.now()
fromDate=datetime(currentDate.year,currentDate.month-3, currentDate.day)
print(fromDate)
histryData=breeze.get_historical_data_v2(interval="1day",
                            from_date= fromDate.strftime('%Y-%m-%dT%H:%M:%S.000Z'),
                            to_date= currentDate.strftime('%Y-%m-%dT%H:%M:%S.000Z'),
                            stock_code="NIFTY",
                            exchange_code="NSE",
                            product_type="cash",
                            expiry_date="2024-04-25T07:00:00.000Z",
                            right="others",
                            strike_price="0")['Success']

df=pd.DataFrame(histryData)
df.set_index("datetime")
df.drop(['exchange_code','stock_code','volume'],axis=1, inplace=True)

# Calculate EMA 10 fast
df['ema_10']=df['close'].ewm(span=10).mean()

# Calculate EMA 30 slow
df['ema_30']=df['close'].ewm(span=30).mean()

# Calculate MACD
macdDf=ta.macd(close=df['close'],fast=12,slow=26,signal=9)
df['macd']=macdDf['MACD_12_26_9']
df['macd_signal']=macdDf['MACDs_12_26_9']
df['macd_histogram']=macdDf['MACDh_12_26_9']

# Calculate SuperTrend with factor 20,2
superTrenddf=ta.supertrend(high=df['high'],low=df['low'],close=df['close'], multiplier=20,offset=2)
df['superTrend']=superTrenddf['SUPERT_7_20.0']
df['supertrend_signal']=superTrenddf['SUPERTd_7_20.0']
df.to_csv("data.csv",)
df

In [ ]:
def gen_macd_color(df):
    macd_color = []
    macd_color.clear()
    for i in range (0,len(df["MACDh_12_26_9"])):
        if df["MACDh_12_26_9"][i] >= 0 and df["MACDh_12_26_9"][i-1] < df["MACDh_12_26_9"][i]:
            macd_color.append('#26A69A')
            #print(i,'green')
        elif df["MACDh_12_26_9"][i] >= 0 and df["MACDh_12_26_9"][i-1] > df["MACDh_12_26_9"][i]:
            macd_color.append('#B2DFDB')
            #print(i,'faint green')
        elif df["MACDh_12_26_9"][i] < 0 and df["MACDh_12_26_9"][i-1] > df["MACDh_12_26_9"][i] :
            #print(i,'red')
            macd_color.append('#FF5252')
        elif df["MACDh_12_26_9"][i] < 0 and df["MACDh_12_26_9"][i-1] < df["MACDh_12_26_9"][i] :
            #print(i,'faint red')
            macd_color.append('#FFCDD2')
        else:
            macd_color.append('#000000')
            #print(i,'no')
    return macd_color

In [ ]:
macd_color = gen_macd_color(macdDf)
df.index=pd.to_datetime(df.index)

ema10_plot=mpf.make_addplot(data=df['ema_10'],color='b')
ema30_plot=mpf.make_addplot(data=df['ema_30'],color='r')
macd_plot=mpf.make_addplot(data=df['macd'],color='#2962FF',panel=1)
macds_plot=mpf.make_addplot(data=df['macd_signal'],color='#FF6D00', panel=1)
macdh_plot=mpf.make_addplot(data=df['macd_histogram'],type='bar',width=0.7,panel=1, color=macd_color,alpha=1,secondary_y=True)

mpf.plot(df,type='candle', style='charles',volume=False, addplot=[ema10_plot, ema30_plot,macd_plot,macds_plot,macdh_plot])

In [ ]:
# demo strategy, if ema_9 crosses up ema_26, buy
# else if ema_9 crosses down ema_26, sell